# PGY3 Block Schedule for 2019-2020

The schedule covers 12 months: July 1, 2019 to June 30, 2020. There will be 14 PGY3s, including 6 UH, 4 DH, and 4 DH-
M/P residents, who will have slightly different schedules based on the rotations they have already completed.

Morgan will take 7 weeks of maternity leave and graduate 7 weeks later than others.

In [1]:
MONTHS = ['Jul-19', 'Aug-19', 'Sep-19', 'Oct-19',
          'Nov-19', 'Dec-19', 'Jan-20', 'Feb-20',
          'Mar-20', 'Apr-20', 'May-20', 'Jun-20',
          'Jul-20', 'Aug-20', # Extra months for Morgan
         ]

MORGAN_MISSING_MONTHS = ['Jul-19', 'Aug-19']
MORGAN_EXTRA_MONTHS = ['Jul-20', 'Aug-20']

UH_RESIDENTS = ['Alisa', 'Cristi', 'Jeff', 'Naomi', 'Steve', 'Kenny']
DH_RESIDENTS = ['Alicia', 'Anita', 'John', 'Morgan']

DH_M_RESIDENTS = ['Leslie', 'Sydney']
DH_P_RESIDENTS = ['Ashley', 'Shalvi']
DH_MP_RESIDENTS = DH_M_RESIDENTS + DH_P_RESIDENTS
RESIDENTS = UH_RESIDENTS + DH_RESIDENTS + DH_MP_RESIDENTS

INPATIENT_ROTATIONS = ['AFM', 'FMS', 'St-Anthonys']
VACATIONABLE_ROTATIONS = [
    'Geriatrics', 'Derm', 'PTL', 'Outpatient-Peds-for-DH/MP',
    'Practice-Management', 'Elective', 'Behavioral-Health',
    'MSK-2-Denver', 'MSK-2-Winter-Park',
]
ROTATIONS = VACATIONABLE_ROTATIONS + INPATIENT_ROTATIONS + [
    'Rural', 'Cardiology',
]

In [2]:
import citrus
import pulp
from citrus import negate
from functools import reduce
import pandas as pd
from itertools import product
model = citrus.Problem('pgy3-2019-schedule', pulp.LpMinimize)

x = model.dicts(
    'x',
    ((month, rotation, resident)
     for month in MONTHS
     for rotation in ROTATIONS
     for resident in RESIDENTS),
    cat=pulp.LpBinary,
)


PROGRAM_OBJECTIVE = []

def avg(lst):
    """
    why not just sum(lst) / len(lst)?
    That iterates over lst twice. Possible that
    lst is actually a generator, which can only be
    consumed once.
    """
    total = 0
    num = 0
    for item in lst:
        total += item
        num += 1
    return total / num


def inpatient_during(month, resident):
    return reduce(citrus.logical_or, [x[month, r, resident] for r in INPATIENT_ROTATIONS])

def no_two_inpatient_in_a_row(resident):
    sequential_inpatient = []
    for m1, m2 in zip(MONTHS, MONTHS[1:]):
        m1_is_inpatient = inpatient_during(m1, resident)
        m2_is_inpatient = inpatient_during(m2, resident)
        both_are_inpatient = m1_is_inpatient & m2_is_inpatient
        sequential_inpatient.append(negate(both_are_inpatient))
    # avg rather than sum below because each of
    # "NOT (Jan inpatient & Feb inpatient)" is a request
    return avg(sequential_inpatient)


## "No Time Turners" Constraints

In [3]:
# No time-turner constraints, and Morgan's offset months
for resident in RESIDENTS:
    missing_months = MORGAN_MISSING_MONTHS if resident == 'Morgan' else MORGAN_EXTRA_MONTHS
    res_months = [m for m in MONTHS if not m in missing_months]
    assert len(res_months) == 12
    assert len(missing_months) == 2
    for month in res_months:
        model.addConstraint(
            sum(x[month, rotation, resident] for rotation in ROTATIONS) == 1,
            f'{resident} must do exactly one rotation during {month}'
        )
    for month in missing_months:
        model.addConstraint(
            sum(x[month, rotation, resident] for rotation in ROTATIONS) == 0,
            f'{resident} must do exactly zero rotation during {month}'

        )

## Curriculum Constraints

### Each UH/DH PGY3 resident must have the following rotations:
- One month each of MSK II, Derm, Cardiology, Geriatrics, and Practice Transformation Leadership (PTL)
- Either three elective months (DH) or two electives/one rural rotation (UH)

In [4]:
for resident in (UH_RESIDENTS + DH_RESIDENTS):
    for rotation in ('Derm', 'Cardiology', 'Geriatrics', 'PTL', 'MSK-2-Winter-Park'):
        model.addConstraint(
            sum(x[m, rotation, resident] for m in MONTHS) == 1,
            f'{resident} must do 1 month of {rotation}',
        )

for resident in UH_RESIDENTS:
    model.addConstraint(
        sum(x[m, r, resident] for m in MONTHS for r in ('Elective', 'St-Anthonys')) == 2,
        f'{resident} must do 2 months of Elective/St. As',
    )
    model.addConstraint(
        sum(x[m, 'Rural', resident] for m in MONTHS) == 1,
        f'{resident} must do 1 month of Rural',
    )
    model.addConstraint(
        sum(x[m, r, resident] for m in MONTHS for r in ('Practice-Management', 'Outpatient-Peds-for-DH/MP', 'Behavioral-Health', 'MSK-2-Denver')) == 0,
        f'{resident} must do 0 months of PTM, Outpatient Peds, MSK Denver',
    )


for resident in DH_RESIDENTS:
    model.addConstraint(
        sum(x[m, r, resident] for m in MONTHS for r in ('Elective', 'St-Anthonys')) >= 3,
        f'{resident} must do at least 3 months of Elective/St. As',
    )
    model.addConstraint(
        sum(
            x[m, r, resident]
            for m in MONTHS
            for r in (
                'Rural', 'Practice-Management', 'Outpatient-Peds-for-DH/MP',
                'Behavioral-Health', 'MSK-2-Denver'
            )
        ) == 0,
        f'{resident} must do 0 months of Rural, PTM, Outpatient Peds, MSK Denver',
    )


### Each DH-M/P PGY3 resident must have the following rotations:
- One month each of MSK II, Dermatology, Geriatrics, Practice Management, Outpatient Peds for DH-M/P, and the
Behavioral Health rotation with Alex Reed
- Three months of electives

In [5]:
for resident in DH_MP_RESIDENTS:
    for rotation in ('Derm', 'Geriatrics', 'Practice-Management', 'Outpatient-Peds-for-DH/MP', 'Behavioral-Health', 'MSK-2-Denver'):
        model.addConstraint(
            sum(x[m, rotation, resident] for m in MONTHS) == 1,
            f'{resident} must do 1 month of {rotation}'
        )
    model.addConstraint(
        sum(x[m, 'Elective', resident] for m in MONTHS) == 3,
        f'{resident} must do 3 months of Elective',
    )

### FMS/AFM:
- UH R3s will all need to complete 4 months of inpatient total. Three of you must complete 1 month of AFM and 3 months of FMS; the other three will do 4 months of FMS.
- DH and DH-MP Residents must do the following numbers of each

| R3     | FMS | AFM | Notes                                         |
|--------|-----|-----|-----------------------------------------------|
| Alicia | 0   | 3   | December, Feb or March, and May are suggested |
| Anita  | 3   | 1   |                                               |
| John   | 3   | 1   |                                               |
| Morgan | 3   | 1   |                                               |
| Leslie | 2   | 1   |                                               |
| Sydney | 3   | 0   |                                               |
| Ashley | 2   | 1   |                                               |
| Shalvi | 2   | 1   |                                               |

In [6]:
for resident in UH_RESIDENTS:
    model.addConstraint(
        sum(x[m, r, resident] for m in MONTHS for r in ('AFM', 'FMS')) == 4,
        f'{resident} must do 4 total months of AFM+FMS',
    )
    model.addConstraint(
        sum(x[m, 'AFM', resident] for m in MONTHS) <= 1,
        f'{resident} must do not more than 1 month of AFM',
    )

fms_afm_numbers = {
    'Alicia': (0, 3),
    'Anita': (3, 1),
    'John': (3, 1),
    'Morgan': (3, 1),
    'Leslie': (2, 1),
    'Sydney': (3, 0),
    'Ashley': (2, 1),
    'Shalvi': (2, 1),
}
for resident, (fms_count, afm_count) in fms_afm_numbers.items():
    model.addConstraint(
        sum(x[m, 'FMS', resident] for m in MONTHS) == fms_count,
        f'{resident} must do {fms_count} months of FMS',
    )
    model.addConstraint(
        sum(x[m, 'AFM', resident] for m in MONTHS) == afm_count,
        f'{resident} must do {afm_count} months of AFM',
    )
    
# As preferences, add the notes for alicia's schedule
# PROGRAM_OBJECTIVE.append(avg([
#     x['Dec-19', 'AFM', 'Alicia'],
#     x['Feb-20', 'AFM', 'Alicia'] | x['Mar-20', 'AFM', 'Alicia'],
#     x['May-20', 'AFM', 'Alicia'],
# ]))
model.addConstraint(
    no_two_inpatient_in_a_row('Alicia') == 1,
    'Alicia should not do two months of inpatient in a row'
)

### Practice Transformation Leadership (PTL) or Practice Management
- UH/DH residents should complete this rotation from July to December so we have persistent leadership for our class QI project that is due to the ABFM in December.
    - You may have one DH and one UH resident on at the same time, but not two people from the same practice.

- DH-M/P residents will complete the Practice Management rotation between August-December, supervised by Nida. Residents can do this different months or the two residents at each clinic site can do it during the same month.

In [7]:
for month in MONTHS:
    if month in ('Jul-19', 'Aug-19', 'Sep-19', 'Oct-19', 'Nov-19', 'Dec-19'):
        for residents, group_name in ((DH_RESIDENTS, 'DH'), (UH_RESIDENTS, 'UH')):
            model.addConstraint(
                sum(x[month, 'PTL', r] for r in residents) <= 1,
                f'No more than one resident from {group_name} may do PTL during {month}',
            )
    else: 
        model.addConstraint(
            sum(x[month, 'PTL', r] for r in RESIDENTS) == 0,
            f'No one should do PTL during {month}',
        )
    
for month in MONTHS:
    if month in ('Aug-19', 'Sep-19', 'Oct-19', 'Nov-19', 'Dec-19'):
        for residents, group_name in ((DH_M_RESIDENTS, 'Montbello'), (DH_P_RESIDENTS, 'Park-Hill')): 
            model.addConstraint(
                sum(x[month, 'Practice-Management', r] for r in residents) <= 1,
                f'No more than one resident from {group_name} may do Practice-Management during {month}',
            )
    else:
        model.addConstraint(
            sum(x[month, 'Practice-Management', r] for r in RESIDENTS) == 0,
            f'No one should do Practice Management during {month}',
        )

## Staffing Constraints

### Cards, Derm, Geri:
- Cardiology: Needs to be done by DH and UH residents only. Only one per month.
- Dermatology: Needs to be done by DH, UH, and DH-M/P residents. You should assign one person per month except for two months when you can have two people.
- Geriatrics: Needs to be done by DH, UH, and DH-M/P residents. You should assign one person per month except for two months when you can have two people.
    - Brigitte Utter will be doing Geri in August, so only schedule 1 additional learner in August.

In [8]:
for month in MONTHS:
    model.addConstraint(
        sum(x[month, 'Cardiology', r] for r in DH_RESIDENTS + UH_RESIDENTS) <= 1,
        f'No more than one resident on Cardiology during {month}',
    )

for rotation in ('Derm', 'Geriatrics'):
    for month in set(MONTHS) - set(MORGAN_EXTRA_MONTHS):
        model.addConstraint(
            sum(x[month, rotation, r] for r in RESIDENTS) >= 1,
            f'At least one resident on {rotation} during {month}'
            # This is sort of a backwards way of requiring that most months should have one,
            # two months may have two, people on Derm. Combined with the curriculum constraint
            # that each resident must do derm exactly once, this accomplishes the goal.
        )

model.addConstraint(
    sum(x['Aug-19', 'Geriatrics', r] for r in RESIDENTS) <= 1,
    f'No more than one resident may do Geriatrics during August',
)

### DH-M/P Only Rotations
- Outpatient Peds: You should only have one resident on Outpatient Peds per month between July-January. Vacation is allowed during the month.
- Behavioral Health rotation: One resident per month, any month except July.

In [9]:
model.addConstraint(
    sum(x[month, rotation, resident]
        for month in MONTHS
        for rotation in ('Outpatient-Peds-for-DH/MP', 'Behavioral-Health')
        for resident in DH_RESIDENTS + UH_RESIDENTS
       ) == 0,
    f'Only DH_MP residents should do Outpatient Peds or Behavioral Health',
)
for month in MONTHS:
    if month in ('Jul-19', 'Aug-19', 'Sep-19', 'Oct-19', 'Nov-19', 'Dec-19', 'Jan-20'):
        model.addConstraint(
            sum(x[month, 'Outpatient-Peds-for-DH/MP', r] for r in DH_MP_RESIDENTS) <= 1,
            f'No more than one resident on Outpatient-Peds-for-DH/MP during {month}',
        )
    else:
        model.addConstraint(
            sum(x[month, 'Outpatient-Peds-for-DH/MP', r] for r in RESIDENTS) == 0,
            f'No one should be on Outpatient-Peds-for-DH/MP during {month}',
        )
        
    if month == 'Jul-19':
        model.addConstraint(
            sum(x[month, 'Behavioral-Health', r] for r in RESIDENTS) == 0,
            f'No one should be on Behavioral-Health during {month}',
        )
    else:
        model.addConstraint(
            sum(x[month, 'Behavioral-Health', r] for r in DH_MP_RESIDENTS) <= 1,
            f'No more than one resident on Behavioral-Health during {month}',
        )

### FMS/AFM Staffing Constraints

|    &nbsp; | Jul-19 | Aug-19 | Sep-19 | Oct-19 | Nov-19 | Dec-19 | Jan-20 | Feb-20 | Mar-20 | Apr-20 | May-20 | Jun-20 |
|-----|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|--------|
| FMS | 2      | 3      | 3      | 4      | 3      | 3      | 4      | 4      | 3      | 3      | 3      | 4      |
| AFM | 0      | 0      | 0      | 0      | 0      | 2      | 2      | 2      | 2      | 2      | 2      | 0      |

In [10]:
fms_afm_staffing_numbers = {
    'Jul-19': (2, 0),
    'Aug-19': (3, 0),
    'Sep-19': (3, 0),
    'Oct-19': (4, 0),
    'Nov-19': (3, 0),
    'Dec-19': (3, 2),
    'Jan-20': (4, 2),
    'Feb-20': (4, 2),
    'Mar-20': (3, 2),
    'Apr-20': (3, 2),
    'May-20': (3, 2),
    'Jun-20': (4, 0),
}

for month, (fms_count, afm_count) in fms_afm_staffing_numbers.items():
    model.addConstraint(
        sum(x[month, 'FMS', r] for r in RESIDENTS) == fms_count,
        f'{fms_count} people must staff FMS during {month}',
    )
    model.addConstraint(
        sum(x[month, 'AFM', r] for r in RESIDENTS) == afm_count,
        f'{afm_count} people must staff AFM during {month}',
    )

### MSK II (Winter Park or Denver):
- Residents complete this rotation at Winter Park or in Denver.
- The four DH-M/P residents will complete the rotation in Denver in July or January through June. They should all be scheduled for one month, though the amount of time they spend in the rotation will vary.
    - Ashley, Leslie, Shalvi will all do one month of MSK II, while Sydney will do 2 weeks and spend the rest of the month in continuity clinic and/or vacation. They may request specific months based on their individual plans for the rotation, which we have emailed about separately.
- UH and DH residents cover the Winter Park practice site from the middle of November through the middle of April, as well as the month of August. This is how many residents we need each month:
    - August: 2 residents
    - November: 1 resident (last two weeks of the month)
    - December: 2 residents
    - January: 2 residents
    - February: 1 resident
    - March: 1 resident
    - April: 1 resident (first two weeks of the month)

In [11]:
for month in ('Jul-19', 'Jan-20', 'Feb-20',
          'Mar-20', 'Apr-20', 'May-20', 'Jun-20'):
    model.addConstraint(
        sum(x[month, 'MSK-2-Denver', r] for r in DH_MP_RESIDENTS) <= 1,
        f'No more than one resident on MSK-2 in Denver during {month}'
    )

winter_park_msk = {
    'Aug-19': 2,
    'Nov-19': 1,
    'Dec-19': 2,
    'Jan-20': 2,
    'Feb-20': 1,
    'Mar-20': 1,
    'Apr-20': 1,
}
for month, msk_num in winter_park_msk.items():
    model.addConstraint(
        sum(x[month, 'MSK-2-Winter-Park', r] for r in UH_RESIDENTS + DH_RESIDENTS) == msk_num,
        f'{msk_num} residents must be on MSK-2 at Winter Park in {month}'
    )

## Confirmed Away Rotations

These rotations are coordinated with St. Anthony's, and must be treated like constraints

- October- Cristi
- November- Alisa
- December- Morgan
- January-Naomi
- February- John

In [12]:
st_antys_combos = (
    ('Morgan', 'Dec-19'),
    ('Alisa', 'Nov-19'),
    ('Cristi', 'Oct-19'),
    ('Naomi', 'Jan-20'),
    ('John', 'Feb-20'),
)
for resident, month in product(RESIDENTS, MONTHS):
    if (resident, month) in st_antys_combos:
        model.addConstraint(
            x[month, 'St-Anthonys', resident] == 1,
            f"{resident} must do St-Anthonys during {month}"
        )
    else:
        model.addConstraint(
            x[month, 'St-Anthonys', resident] == 0,
            f"{resident} must NOT do St-Anthonys during {month}"
        )


## Crispiness Rule

No three inpatient in a row


In [13]:
for resident in RESIDENTS:
    for m1, m2, m3 in zip(MONTHS, MONTHS[1:], MONTHS[2:]):
        m1_is_inpatient = inpatient_during(m1, resident)
        m2_is_inpatient = inpatient_during(m2, resident)
        m3_is_inpatient = inpatient_during(m3, resident)
        model.addConstraint(
            (m1_is_inpatient & m2_is_inpatient & m3_is_inpatient) == 0,
            f'{resident}: {m1}, {m2}, {m3} not all inpatient',
        )

## Cock-blocking Rule

Cristi and John may not rotate together

In [14]:
for rotation in ('MSK-2-Winter-Park',):
    for month in MONTHS:
        model.addConstraint(
            negate(x[month, rotation, 'John']) | negate(x[month, rotation, 'Cristi']) == 1,
            f'John and Cristi may not both be on {rotation} during {month}'
        )

## Resident Preferences

In [15]:
def vacation_during(month, resident):
    # winter park is also vacationable except during nov and apr
    rotations = VACATIONABLE_ROTATIONS[:]
    if month in ('Dec-19', 'Apr-20'):
        rotations.remove('MSK-2-Winter-Park')
        
    return reduce(citrus.logical_or, [x[month, r, resident] for r in rotations])

def as_early_as_possible(resident, rotation, months):
    divisor = len(MONTHS) - 1
    weights = [w / divisor for w in reversed(range(len(MONTHS)))]
    # weights is [ 11/11, 10/11, ... 1/11, 0/11]
    return sum(w * x[month, rotation, resident] for w, month in zip(weights, MONTHS))

resident_prefs = {}

### Jeff's Preferences

1. Vacationable, non-elective month in January
2. Would like to senior AFM.
3. Vacationable month in October

In [16]:
resident_prefs['Jeff'] = (
    3/6 * sum(x['Jan-20', r, 'Jeff'] for r in VACATIONABLE_ROTATIONS if r != 'Elective'),
    2/6 * sum(x[m, 'AFM', 'Jeff'] for m in MONTHS),
    1/6 * sum(x['Oct-19', r, 'Jeff'] for r in VACATIONABLE_ROTATIONS),
)

### Leslie's Preferences

So I really have just requests/3 months that I can't be on inpatient  (AFM/FMS)

1. October- sister's wedding (which I have to take off at least 2 days)
2. December - family vacation planned already for Christmas
3. March- vacation with husband (who is a radiology resident at CU so he requested that month off too. Vacation not yet planned officially unless we both get that month off)
4. MSK-2-Denver during Jul-19, Aug-19, or Sep-19

In [17]:
resident_prefs['Leslie'] = (
    4/10 * negate(inpatient_during('Oct-19', 'Leslie')),
    3/10 * negate(inpatient_during('Dec-19', 'Leslie')),
    2/10 * negate(inpatient_during('Mar-20', 'Leslie')),
    1/10 * sum(x[m, 'MSK-2-Denver', 'Leslie'] for m in ('Jul-19', 'Aug-19', 'Sep-19')),
)

### Anita's Preferences

1. I’d like to be on elective or Geri or Derm in December - no pref, just want to take vacation
2. I’d like to not be on FMS in July (coming off FMS in June)
3. I’d like to not have 2 inpatient months in a row
4. I’d like to be on outpatient in June
5. I’d like to be on outpatient in May
6. I’d like to have Cards in July or August

In [18]:
resident_prefs['Anita'] = (
    6/21 * vacation_during('Dec-19', 'Anita'),
    5/21 * citrus.negate(x['Jul-19', 'FMS', 'Anita']),
    4/21 * negate(inpatient_during('Jun-20', 'Anita')),
    3/21 * negate(inpatient_during('May-20', 'Anita')),
    2/21 * no_two_inpatient_in_a_row('Anita'),
    1/21 * sum(x[m, 'Cardiology', 'Anita'] for m in ('Jul-19', 'Aug-19')),
)

### Sydney's Preferences

1. I would like to be on an elective in August (or some other rotation where I could take a week off). (I'm getting married August 31st and was hoping to take a week off before to prepare.)
2. I would like to be on a rotation in September where I could take a few days off. (The wedding is labor day weekend and I was hoping to take the first Monday in September off to hang out with some of the people who are flying in).
3.  I would like an elective / other vacationable month in February.
4. I would like to do Sports Medicine in Jan-March (if at all possible).

In [19]:
model.addConstraint(vacation_during('Aug-19', 'Sydney') == 1, 'Sydney is getting married in August')
resident_prefs['Sydney'] = (
#     4/10 * vacation_during('Aug-19', 'Sydney'),
    3/6 * vacation_during('Sep-19', 'Sydney'),
    2/6 * vacation_during('Feb-20', 'Sydney'),
    1/6 * sum(x[m, 'MSK-2-Denver', 'Sydney'] for m in ('Jan-20', 'Feb-20', 'Mar-20')),
)

### Kenny's Preferences

1. Elective in December (there's an elective I'd like to do that's only available in December)
2. Any vacationable month in July (geriatrics, elective, etc)

In [20]:
resident_prefs['Kenny'] = (
    2/3 * x['Dec-19', 'Elective', 'Kenny'],
    1/3 * vacation_during('Jul-19', 'Kenny')
)

### Morgan's Preferences

1. No two inpatient months in a row (incl. St. Anthony's in Dec)
2. Vacationable Month in September
3. Winter park during Nov, Feb, Mar, or Apr
4. FMS during Jul-20

In [21]:
resident_prefs['Morgan'] = (
    2/3 * no_two_inpatient_in_a_row('Morgan'),
    1/3 * vacation_during('Sep-19', 'Morgan'),
)

### Alisa's Preferences

1. Outpatient in July to take vacation for wedding and reunion
2. Alternate inpatient and outpatient rotations as much as possible (incl. St. A's in Nov-19)
3. Do away rotations (winter park and rural) earlier in the academic year
4. December: outpatient
5. May: outpatient

In [22]:
resident_prefs['Alisa'] = (
    5/15 * negate(inpatient_during('Jul-19', 'Alisa')),
    4/15 * no_two_inpatient_in_a_row('Alisa'),
    3/15 * avg(
        as_early_as_possible('Alisa', 'Rural', [m for m in MONTHS if m not in MORGAN_EXTRA_MONTHS]) +
        as_early_as_possible('Alisa', 'MSK-2-Winter-Park', [m for m in MONTHS if m not in MORGAN_EXTRA_MONTHS])
    ),
    2/15 * negate(inpatient_during('Dec-19', 'Alisa')),
    1/15 * negate(inpatient_during('May-20', 'Alisa')),
)

### Shalvi's Preferences

1. Elective or MSK-Denver in Sep-19
2. vacationable in Jan-20, Feb-20, Apr-20
3. FMS in July
4. Inpatient in Oct-19, Dec-19, Mar-20

In [23]:
resident_prefs['Shalvi'] = (
    4/10 * sum(x['Sep-19', r, 'Shalvi'] for r in ('Elective', 'MSK-2-Denver')),
    3/10 * avg(vacation_during(m, 'Shalvi') for m in ('Jan-20', 'Feb-20', 'Apr-20')),
    2/10 * x['Jul-19', 'FMS', 'Shalvi'],
    1/10 * avg(inpatient_during(m, 'Shalvi') for m in ('Oct-19', 'Dec-19', 'Mar-20')),
)

### Steve's Preferences

1. Flexible months in October and November for fellowship interviews. Linda told me PTL is most flexible and should be one of these months. Derm and/or Geri would work for the other month. Definitely no inpatient. No rural. Cards would not be great either. 
2. Elective in September for an away rotation.
3. Elective in June (so we can prepare to move for a fellowship).

I’d really like to avoid having 2 inpatient months in a row, but if it’s not possible to make the schedule without having to do this, then that’s fine. 


In [24]:
# resident_prefs['Steve'] = (
#     2/3 * avg(sum(x[m, r, 'Steve'] for r in ('Elective', 'Derm', 'PTL', 'Geriatrics', 'MSK-2-Denver', 'Cardiology'))
#                for m in ('Oct-19', 'Nov-19')),
# #     1/3 * no_two_inpatient_in_a_row('Steve')
# )
for month in ('Oct-19', 'Nov-19'):
    model.addConstraint(
        1 == reduce(
            citrus.logical_or,
            [x[month, r, 'Steve'] for r in ('Elective', 'Derm', 'PTL', 'Geriatrics', 'MSK-2-Denver', 'Cardiology')]),
        f'Steve must do one of these during {month}'
    )
model.addConstraint(x['Sep-19', 'Elective', 'Steve'] == 1, 'Steve has an away rotation scheduled already')
model.addConstraint(x['Jun-20', 'FMS', 'Steve'] == 0, 'Steve cannot do FMS in June')

### Ashley's Preferences

1. FMS in July 2019
2. MSK in January
3. PTM December

In [25]:
resident_prefs['Ashley'] = (
    3/6 * x['Jul-19', 'FMS', 'Ashley'],
    2/6 * x['Jan-20', 'MSK-2-Denver', 'Ashley'],
    1/6 * x['Dec-19', 'Practice-Management', 'Ashley'],
)

### Alicia's Preferences

1. Vacationable Sep, Nov


In [26]:
resident_prefs['Alicia'] = (
    avg(vacation_during(m, 'Alicia') for m in ('Sep-19', 'Nov-19')),
)


### Cristi's Preferences

1. No FMS in June (doing fellowship, this is a constraint)
2. August and September as vacationable months
3. December vacationable

In [27]:
resident_prefs['Cristi'] = (
    2/3 * avg(vacation_during(m, 'Cristi') for m in ('Aug-19', 'Sep-19')),
    1/3 * vacation_during('Dec-19', 'Cristi')
)
model.addConstraint(x['Jun-20', 'FMS', 'Cristi'] == 0, 'Cristi cannot do FMS in June')

### John's Preferences

1. September vacationable
2. December vacationable
3. Winter Park in August

In [28]:
resident_prefs['John'] = (
    3/6 * vacation_during('Sep-19', 'John'),
    2/6 * vacation_during('Dec-19', 'John'),
    1/6 * x['Aug-19', 'MSK-2-Winter-Park', 'John'],
)

### Naomi's Preferences

1. vacationable month in september (elective, geriatrics, derm or PTL)
2. vacationable month in july
3. outpatient month in any of nov, dec, feb, apr, june
4. No FMS in may

In [29]:
naomi_vacation_definition = ('Elective', 'Geriatrics', 'Derm', 'PTL')
resident_prefs['Naomi'] = (
    3/6 * sum(x['Sep-19', r, 'Naomi'] for r in naomi_vacation_definition),
    2/6 * sum(x['Jul-19', r, 'Naomi'] for r in naomi_vacation_definition),
    1/6 * reduce(citrus.logical_or, [negate(inpatient_during(m, 'Naomi')) for m in ('Nov-19', 'Dec-19', 'Feb-20', 'Apr-20', 'Jun-20')]),
#     1/10 * negate(x['May-20', 'FMS', 'Naomi']),
)

## Fairness Objective

In [30]:
RESIDENT_OBJECTIVES = [sum(v) for k, v in resident_prefs.items()]

def minimum(*xs, name):
    y = model.make_var(name, cat=pulp.LpContinuous)
    for x in xs:
        model.addConstraint(y <= x, 'constraint{}'.format(model._synth_var()))
    model.addConstraint(y >= 0, 'constraint{}'.format(model._synth_var()))
    return y

In [31]:
fairness_objective = 10 * minimum(*RESIDENT_OBJECTIVES, name='least satisfied resident goals')

objective = (
    sum(PROGRAM_OBJECTIVE) +
    sum(RESIDENT_OBJECTIVES) +
    fairness_objective
)

## Indicator variables to show resident outcomes

In [32]:
resident_outcomes = {}
for resident, objectives in resident_prefs.items():
    resident_outcomes[resident] = []
    for ix, objective in enumerate(objectives):
        var = model.make_var(name=f'outcome_{resident}_{ix + 1}')
        model.addConstraint(var == objective)
        resident_outcomes[resident].append(var)

## Set everyone's first priority request as a constraint

(If it doesn't cause model to be infeasible)

In [33]:
for resident, indicator_vars in resident_outcomes.items():
    model.addConstraint(
        indicator_vars[0] >= 0.22,
        f'{resident} first choice'
    )

## Moment of truth!

In [34]:
model.setObjective(objective)
model.solve()
if pulp.LpStatus[model.status] != 'Optimal':
    for name, c in model.constraints.items():
        if not c.valid(0):
            print(c.name)
    raise ValueError(pulp.LpStatus[model.status])

## Display Schedules

In [35]:
data = []
for resident in RESIDENTS:
    for month in MONTHS:
        rotations = [(x[month, r, resident].varValue, r) for r in ROTATIONS]
        if any(v is not None and v > 0 for (v, r) in rotations):
            v, r = max(
                ((v, r) for (v,r) in rotations if v is not None),
                key=lambda t: t[0])
        else:
            r = ''
            
        data.append({
            'resident': resident,
            'month': month,
            'rotation': r,
        })


In [36]:
by_resident = (pd.DataFrame.from_records(data)
        .pivot(
        values='rotation',
        index='resident',
        columns='month')
[MONTHS])
by_resident

month,Jul-19,Aug-19,Sep-19,Oct-19,Nov-19,Dec-19,Jan-20,Feb-20,Mar-20,Apr-20,May-20,Jun-20,Jul-20,Aug-20
resident,,,,,,,,,,,,,,
Alicia,Elective,Geriatrics,Cardiology,PTL,MSK-2-Winter-Park,AFM,Elective,AFM,Elective,Elective,AFM,Derm,,
Alisa,Geriatrics,Cardiology,Derm,FMS,St-Anthonys,PTL,MSK-2-Winter-Park,Elective,FMS,FMS,Rural,FMS,,
Anita,Elective,MSK-2-Winter-Park,PTL,FMS,Elective,Elective,FMS,Derm,AFM,FMS,Geriatrics,Cardiology,,
Ashley,FMS,Derm,Practice-Management,Behavioral-Health,Outpatient-Peds-for-DH/MP,AFM,FMS,Elective,MSK-2-Denver,Geriatrics,Elective,Elective,,
Cristi,Rural,MSK-2-Winter-Park,FMS,St-Anthonys,PTL,FMS,Cardiology,Elective,AFM,Derm,FMS,Geriatrics,,
Jeff,Derm,FMS,Rural,PTL,Elective,FMS,Geriatrics,FMS,MSK-2-Winter-Park,Elective,Cardiology,FMS,,
John,FMS,PTL,Elective,FMS,Derm,Cardiology,FMS,St-Anthonys,Geriatrics,MSK-2-Winter-Park,AFM,Elective,,
Kenny,Cardiology,FMS,PTL,Rural,FMS,Elective,Derm,MSK-2-Winter-Park,FMS,Geriatrics,Elective,FMS,,
Leslie,Outpatient-Peds-for-DH/MP,Practice-Management,FMS,MSK-2-Denver,FMS,Derm,AFM,Geriatrics,Elective,Elective,Behavioral-Health,Elective,,


In [37]:
by_rotation = (pd.pivot_table(
        pd.DataFrame.from_records(data),
        values='resident',
        index='rotation',
        columns='month',
        aggfunc=', '.join)
        .fillna('')
[MONTHS])


## Check how we did

In [39]:
pref_results = pd.DataFrame({
    r: [o[ix].varValue if ix < len(o) else None for ix in range(7)]
    for r, o in resident_outcomes.items()
}).T
pref_results['Total'] = pref_results.sum(axis=1)
pref_results

,0,1,2,3,4,5,6,Total
Jeff,0.500000,0.000000,0.166667,NaN,NaN,NaN,NaN,0.666667
Leslie,0.400000,0.300000,0.200000,0.000000,NaN,NaN,NaN,0.900000
Anita,0.285714,0.238095,0.175824,0.142857,0.095238,0.0,NaN,0.937729
Sydney,0.500000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.500000
Kenny,0.666667,0.000000,NaN,NaN,NaN,NaN,NaN,0.666667
Morgan,0.615385,0.000000,NaN,NaN,NaN,NaN,NaN,0.615385
Alisa,0.333333,0.225641,0.015385,0.133333,0.066667,NaN,NaN,0.774359
Shalvi,0.400000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.400000
Ashley,0.500000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.500000
Alicia,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,0.500000
